# 2. Notebook Etapa 2 Automatizacion y Persistencia del Modelo TD-IDF con SVM

In [18]:
import numpy as np
import pandas as pd
import nltk
import openpyxl

import re, string, unicodedata
import contractions
import inflect
from joblib import dump, load
from sklearn.base import BaseEstimator,TransformerMixin


from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import (
    ConfusionMatrixDisplay, RocCurveDisplay,
    roc_auc_score, precision_score, recall_score, f1_score
)
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline

from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from skopt import BayesSearchCV
from skopt.space import Real, Integer


from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.base import BaseEstimator, ClassifierMixin

In [19]:
import spacy
nlp = spacy.load("es_core_news_md")  # Cargar el modelo de spaCy en español

Para este caso lo primero que vamos a realizar es una importancion de los modelos necesarios para poder realizar la division de los textos. En este caso, se sabe que la mayoria de librerias se encuentran en ingles. Por esta razon, es necesario realizar y encontrar las librerias spaCy del modelo, para lograr realizar el analisis de textos en español.

In [20]:
import warnings
warnings.filterwarnings("ignore")

En este punto, es importatnte hacer caso omiso a las alertas en python.

In [21]:
pd.set_option("display.max_colwidth", None)

Para este caso, solo es necesario hacer que para la impresion se toque los outputs de python para poder ver todo el contenido de las celdas, ya que se esta hablando de opiniones un poco largas.

In [22]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("averaged_perceptron_tagger") 

stop_words = stopwords.words("spanish")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Samue\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Samue\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Samue\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


En este punto se esta realizando, ya algo mas profundo en terminos del desarrollo del analisis de los textos. Para este punto, lo que se esta haciendo es defnir esa palabras que realizan un cierto aporte en el analisis, sobre los textos. En este caso, como la mayoria de librerias son desarrolladas en ingles es necesario realizar la busqueda o adaptacion de estas mismas a español

In [23]:
stop_words

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

In [24]:
# Uso de la libreria pandas para la lectura de archivos
data = pd.read_excel('./data/cat_345.xlsx')
# Asignación a una nueva variable de los datos leidos
df_obj=data


En este punto, lo unico que se esta haciendo es la lectura del archivo, para poder cargar la informacion en un dataframe de la libreria de pandas

#### 1. Preparacion de datos

In [25]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    new_words = []
    for word in words:
        new_words.append(word.lower())
    return new_words
    """Convert all characters to lowercase from list of tokenized words"""

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    stop_words = set(stopwords.words("spanish"))
    new_words = [word for word in words if word not in stop_words]
    return new_words

def preprocessing(words):
    words = contractions.fix(words)
    words = word_tokenize(words)
    words = to_lowercase(words)
    words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    return words

class Transformaciones(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        X_copy['Textos_Tokenizados'] = X_copy['Textos_espanol'].apply(preprocessing) 
        return X_copy

En este punto, se definen las funciones o transformaciones necesarias para analizar el texto. En primer lugar, se llevará a cabo una transformación relacionada con el formato ASCII. Esta transformación tiene como objetivo eliminar símbolos o caracteres que no estén en formato ASCII o que no se asemejen a él. Luego, para evitar comparaciones entre palabras idénticas pero con diferencias en mayúsculas y minúsculas, se realizará una transformación que convertirá todo el texto a minúsculas, lo que facilitará el análisis. Posteriormente, se eliminarán los puntos, comas y punto y comas, con el propósito de que el texto sea lo más puro posible y se enfoque exclusivamente en el contenido. Después, se abordará la necesidad de suprimir los números, sustituyéndolos por palabras, con el fin de que no influyan en el análisis. Por último, se abordará el concepto previamente mencionado de "stopwords". En este caso, se eliminarán estas palabras recurrentes que no aportan al análisis y pueden volverlo denso.

In [26]:
df_obj['Textos_espanol'] = df_obj['Textos_espanol'].apply(contractions.fix) #Aplica la corrección de las contracciones

df_obj['Textos_Tokenizados'] = df_obj['Textos_espanol'].apply(preprocessing)  # Aplica la eliminación del ruido

En este punto, se aplica las transformaciones realizadas anteriormente.

#### 2. Division de datos

In [27]:
# The param 'stratify' is useful to guarantee label proportions on train and test sets
X_train, X_test, y_train, y_test = train_test_split(df_obj[["Textos_espanol"]], df_obj["sdg"], test_size=0.3, stratify=df_obj["sdg"], random_state=1)

Para este paso lo que se etsa reliznado es l division de los dato sentre datos de entrenamiento y datos de prueba. Sin embargo, en este caso los dato sde prueba no se avan a utilizar, dado qeu ya el modelo fue evaluaddo y seleccioando en la aprte anteriro se necesitan solo los datos de prueba para poder realizar el entrenamiento del modleo a usar para las predicicones futuras.

#### 3. Transformaciones

En este punto, se esta creadno la calse para facilitar o dar mas entendimiento a la etapa de trasnofmrqciones. En este caso, estas transformaciones. Nose vana guardar en la pipline. Esto se dbe a auq ese estan realizadno con un proceso qeu resulta presentar particularidades a la hora e guardar sus funcionaldiades en la piplien. Por ende, esta etap ade trasnofmraciones es alg qeu se le va aplciar a los modelos o a los datos  procesar futuramente ede manera manul, si se quiere pensar. Sin embargo, esta manualidad, no esta de etsa manera, dado que dentro de la msiam calse, como se habia visto anteriormnet,e existe la funcionalidad de processing, la cua  nos permite realziar un tipo de pipline manaul, para que los datos que entren se les apliqeun todas las trasnfomraciones necesarias.

In [28]:
class TransformacionesPD(BaseEstimator, TransformerMixin):

    def remove_non_ascii(words):
        """Remove non-ASCII characters from list of tokenized words"""
        new_words = []
        for word in words:
            new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_words.append(new_word)
        return new_words

    def to_lowercase(words):
        new_words = []
        for word in words:
            new_words.append(word.lower())
        return new_words
        """Convert all characters to lowercase from list of tokenized words"""

    def remove_punctuation(words):
        """Remove punctuation from list of tokenized words"""
        new_words = []
        for word in words:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
        return new_words

    def replace_numbers(words):
        """Replace all interger occurrences in list of tokenized words with textual representation"""
        p = inflect.engine()
        new_words = []
        for word in words:
            if word.isdigit():
                new_word = p.number_to_words(word)
                new_words.append(new_word)
            else:
                new_words.append(word)
        return new_words

    def remove_stopwords(words):
        stop_words = set(stopwords.words("spanish"))
        new_words = [word for word in words if word not in stop_words]
        return new_words

    def preprocessing(words):
        words = contractions.fix(words)
        words = word_tokenize(words)
        words = to_lowercase(words)
        words = replace_numbers(words)
        words = remove_punctuation(words)
        words = remove_non_ascii(words)
        words = remove_stopwords(words)
        return words

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        X_copy['Textos_Tokenizados'] = X_copy['Textos_espanol'].apply(preprocessing)
        return X_copy

En este punto, lo uncio separado qeu se esta haciendo, es añadiendo las funcionalidades de trasnform y fit. Donde la priemera es la qeu nos va apermitir porcesar de manera autmatica los atos d entrada, para qu se les paliqeun todas las trasnofmraciones necessarias. Luego, fit es este punto no va a realziar ninguna accion, solo retorna el msimo dato.

#### 4. Pipeline

En este paso, vamos a realizar la creacion de la pipeline. La cual nos va apermitir desarrollar la prediccion de los datos de manera casi automatica. En este punto se va aentrenar al modelo se vana realziar las transformaciones requeridas. Luego, se va a guarar, para lograr perisitir este modelo par aluego poder utilzarlo.

In [29]:
transformacionesPD = TransformacionesPD()

tfidf = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stop_words, lowercase=True)

X_tfidf = tfidf.fit_transform(X_train["Textos_espanol"])

svm_model = SVC(kernel='linear', random_state=5)

svm_model.fit(X_tfidf, y_train)

pipeline = Pipeline([
    ("tfidf", tfidf),
    ("svm_model", svm_model)
])

filename= "tratamientoD.joblib"

dump(pipeline,filename)


['tratamientoD.joblib']

Para este caos lo que se realizo fue. Primero la creacion del modelo TD-IDF. El cual no svecotirza lso textos, de maner auqe contiene una frecuencia de aparicion de las palabras, qeu fue explicado en la etapa aneriro. Luego, se realiza el entrenamiento de este modelo con los datos iniciales que se tiene del texto ejemplo o del texto entrenamiento. Luego, se crea el modelo SVM, el cual nose permite desarrollar las prediciones de la variable objeivo. En este caso, desarrolla una seleccion de parametros por medio de la localizacion en un plano espacial de los datos, usnado ideas de hiperplanos entre otras cosas. Luego, se realiza el entrenamiento de los datos que genero el modelo de vectorizacion. Para lograra ya tener entrenados ambos modelo. De esta manera se puede proseguir con la creacion e la pipeline. En este caso,se realzia l apipeline con ambos mdoelso qeu fueron entrenaos anteriomente. Cada uno primer el TD-IDF, y luego el SVM. En este punto, luego de tener la pipeline creada, vamos a desarrollar, la persistencia. En este caso, es hacer el dumpl de la pipline, en un arhcivo joblib. El cual nos va apermitir usarlo despues.

#### 4. Prueba

En este caso se esta realizadno la prueba con los archvios de predicion de la etapa 1. Esto par aporbar el funcionamiento de la automatizacion

In [30]:
filename= "tratamientoD.joblib"

pipeline_loaded = load(filename)

new_data = pd.read_excel('./data/SinEtiquetatest_cat_345 (1).xlsx')

df_transformado = transformacionesPD.transform(new_data)

X_tfidf = pipeline_loaded.named_steps['tfidf'].transform(df_transformado["Textos_espanol"])

predicciones = pipeline_loaded.named_steps['svm_model'].predict(X_tfidf)

df_transformado['sdg'] = predicciones

df_transformado.to_csv('./data/prueba.csv', index=False)


Los datos inciiales son transformados, primero por las trasnofmraciones para poder realizar el modelo. Luego, se importa la pipeline, para lgrar traer onsigo los modelos a ejecutrar. En este punto con la pipeline, ya importada, se procede primero a realizar la ejecucuioon del modelo TD-IDF, para poder realizar la vectorizacion del texto. Luego, se realiza la predicion con el modelo SVM, ambos modelos ya entrenados. En est epunto los resultados se meten en la variable objetivo de los atos, y se almacenadda , el dataframe, en un archivo csv, para que se pueda leer en otro formato.

De esta prueba, se puede sacar, que la automatizacion funciona correctamente, y la prediccion , seria evaluar con datos correctos, pero despues de realizar un analisis un poco prgamtico, s epued decir, que si se esta cumpliendo con la asignacion de los elementos.